In [12]:
import pandas as pd
import numpy as np
import sympy as sp
import matplotlib.pyplot as plt
import sys
import os
from scipy.optimize import minimize

sys.path.append(os.path.dirname(os.getcwd()))

from lib.sympy_helper import normalise
from lib.sampler import generate_reference_data

In [13]:
csv_path = f"../outputs/max-corr-5/hall_of_fame.csv"
equation_df = pd.read_csv(csv_path)
best_equation = equation_df.iloc[-1]['Equation']
sympy_expr = sp.sympify(best_equation)
sympy_expr_clean = normalise(sympy_expr)

In [14]:
def _new_const(counter):
    name = f'c{counter}'
    return sp.Symbol(name)

replacements = {}
default_values = []
constants = []
counter = 0
for node in sp.preorder_traversal(sympy_expr_clean):    
    if isinstance(node, sp.Number) and node not in replacements:
        c = _new_const(counter)
        counter += 1
        replacements[node] = c
        default_values.append(node)
        constants.append(c)

expr_with_constants = sympy_expr_clean.xreplace(replacements)

# TEMP: UNSAFE FIX FOR NEGATIVE BASES
# def is_unsafe_pow(expr):
#     return isinstance(expr, sp.Pow) and not expr.exp.is_integer

# def safe_pow_replacement(expr):
#     return sp.Pow(sp.Abs(expr.base), expr.exp)
# expr_with_constants = expr_with_constants.replace(is_unsafe_pow, safe_pow_replacement)

l = sp.latex(expr_with_constants)
print(l)


\left|{\left(x_{2} \left(c_{0} + c_{1} x_{2} + c_{2}^{c_{3}} x_{2} + c_{4} \left(c_{15} + c_{16} x_{3}^{c_{9}} \left(c_{17} + x_{2}\right)\right) \left(c_{18}^{c_{19}} + x_{3}\right) \left(x_{2} + \left(c_{10} c_{12}^{c_{13}} x_{1}^{c_{11}} + c_{14} x_{1} x_{2}\right) \left|{c_{5} + c_{6} x_{1}^{c_{7}} + c_{8} x_{2}}\right|^{c_{9}}\right)^{c_{9}}\right)^{c_{9}}\right)^{c_{20}}}\right|^{c_{19}}


In [15]:
_vars = ['x1'] + constants

In [16]:
python_fn  = sp.lambdify(_vars, expr_with_constants, modules='numpy')
# (diameter, *constant_values)
python_fn(1e-3, *default_values)

Abs((x2/(1.355900186947505*x2 + (1883.3828630902532 - 8.5108364985610056*(x2 - 44.674122117495778)/x3)*(x3 + 6.2103270637693174)/(x2 + (59.30217941577672 - 0.45347519177667666*x2)/Abs(0.05528710792205269*x2 - 25.076925892914443)) - 101.64243612640218))**0.9427448586559175)**2

In [17]:
df = generate_reference_data(10_000)

def loss_fn(params: np.ndarray):
    """Implements mean squared relative error

    Args:
        params (np.ndarray): input of constants array
    """
    y_pred = python_fn(df['diameter'].values, *params)
    loss = (np.abs(df['v_t'] - y_pred) / df['v_t']) ** 2
    return np.mean(loss)
    # loss = (np.abs(df['v_t'] - y_pred) / df['v_t'])
    # return np.max(loss)

In [ ]:
starting_loss = loss_fn(default_values)

In [ ]:
result = minimize(loss_fn, default_values, method='L-BFGS-B')
final_loss = loss_fn(result.x)

<lambdifygenerated-1>:2: RuntimeWarning: invalid value encountered in power
  return (c0*(c1 + c19*x1**c3 + c2*(c14*(c15 + c16*x1*(c17 + c18*x1)**c3)**c3 + c3*abs(c4 + c5*(c6 + c7*x1**c3)**c3 + c8*x1**c9*(c10 + c11*x1)**c3*(c12 + c13*x1)**c3))**c3)**c3)**c9


In [ ]:
print(starting_loss, final_loss)

7.511155267543588e-06 7.511155267543649e-06
